In [1]:
# Importing libraries
import os
import pandas as pd
import random
import numpy as np
import librosa
import librosa.display
import IPython.display as ipd
from IPython.core.display import display
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
%config InlineBackend.figure_format='retina'
import warnings
warnings.filterwarnings("ignore")


In [2]:
#Getting RAVDESS Data:

paths, labels, duration = [], [], []

for dirname, _, filenames in os.walk('C:/Users/Majid/Programming files/FYP/Datasets/RAVDESS'):
    for filename in filenames:
        
        paths.append(os.path.join(dirname,filename))
        
        duration.append(round(librosa.get_duration(filename=paths[-1]),3))
        
        label = filename[::-1].split('_')[0][::-1]
        
        if label[6:8] == '01':
            labels.append('neutral')
        elif label[6:8] == '02':
            labels.append('calm')
        elif label[6:8] == '03':
            labels.append('happy')
        elif label[6:8] == '04':
            labels.append('sad')
        elif label[6:8] == '05':
            labels.append('angry')
        elif label[6:8] == '06':
            labels.append('fear')
        elif label[6:8] == '07':
            labels.append('disgust')
        elif label[6:8] == '08':
            labels.append('surprise')       

df_ravdess = pd.DataFrame({'path':paths,'duration': duration, 'dataset': 'RAVDESS', 'emotion':labels})

df_ravdess.sample(5)

,path,duration,dataset,emotion
2191,C:/Users/Majid/Programming files/FYP/Datasets/...,3.370,RAVDESS,angry
240,C:/Users/Majid/Programming files/FYP/Datasets/...,3.604,RAVDESS,neutral
2013,C:/Users/Majid/Programming files/FYP/Datasets/...,4.471,RAVDESS,angry
1284,C:/Users/Majid/Programming files/FYP/Datasets/...,3.737,RAVDESS,sad
1320,C:/Users/Majid/Programming files/FYP/Datasets/...,3.804,RAVDESS,neutral


In [3]:
#Getting TESS Data:

paths, labels, duration = [],[],[]

for dirname, _, filenames in os.walk('C:\\Users\\Majid\\Programming files\\FYP\\Datasets\\TESS Toronto emotional speech set data'):
    for filename in filenames:
        
        paths.append(os.path.join(dirname, filename))
        
        duration.append(round(librosa.get_duration(filename=paths[-1]), 3))
        
        label = filename[::-1].split('_')[0][::-1]
        labels.append(label[:-4].lower())

df_tess = pd.DataFrame({'path':paths,'duration': duration, 'dataset': 'TESS', 'emotion':labels})

df_tess['emotion'] = df_tess['emotion'].replace(['ps'], 'surprise')
                  
df_tess.sample(5)

,path,duration,dataset,emotion
122,C:\Users\Majid\Programming files\FYP\Datasets\...,1.576,TESS,angry
3985,C:\Users\Majid\Programming files\FYP\Datasets\...,1.919,TESS,surprise
1138,C:\Users\Majid\Programming files\FYP\Datasets\...,1.918,TESS,surprise
297,C:\Users\Majid\Programming files\FYP\Datasets\...,2.462,TESS,disgust
2542,C:\Users\Majid\Programming files\FYP\Datasets\...,2.224,TESS,surprise


In [4]:
# Getting SAVEE data
paths, labels, duration = [], [], []

for dirname, _, filenames in os.walk('C:\\Users\\Majid\\Programming Files\\FYP\\Datasets\\SAVEE'):
    for filename in filenames:
        
        paths.append(os.path.join(dirname, filename))
        
        label = filename[::-1].split('_')[0][::-1]
        
        if label[:1] == 'a':
            labels.append('angry')
        elif label[:1] == 'd':
            labels.append('disgust')
        elif label[:1] == 'f':
            labels.append('fear')
        elif label[:1] == 'h':
            labels.append('happy')
        elif label[:1] == 'n':
            labels.append('neutral')
        elif label[:1] == 's':
            if label[:2] == 'sa':
                labels.append('sad')
            else:
                labels.append('surprise')

paths = paths[1:] # to filter out 'info.txt' file

for file in paths:
    duration.append(round(librosa.get_duration(filename=file), 3)) 

df_savee = pd.DataFrame({'path':paths, 'duration': duration, 'dataset': 'SAVEE', 'emotion':labels})
                  
df_savee.sample(5)

,path,duration,dataset,emotion
403,C:\Users\Majid\Programming Files\FYP\Datasets\...,3.971,SAVEE,fear
18,C:\Users\Majid\Programming Files\FYP\Datasets\...,4.382,SAVEE,disgust
11,C:\Users\Majid\Programming Files\FYP\Datasets\...,3.315,SAVEE,angry
343,C:\Users\Majid\Programming Files\FYP\Datasets\...,6.080,SAVEE,sad
307,C:\Users\Majid\Programming Files\FYP\Datasets\...,3.544,SAVEE,neutral


In [5]:
# Let's merge the datesets together, now that they have been formatted the same way:

df = pd.concat([df_tess, df_ravdess, df_savee])

# Dropping 'calm & disgust' as out the scope (also not many samples)
df = df[df['emotion'].str.contains('disgust|calm') == False].reset_index(drop=True)

print('The dataset has {} audio files. Below printed 5 random entries:'.format(df.shape[0]))

df.sample(5)

The dataset has 7332 audio files. Below printed 5 random entries:


,path,duration,dataset,emotion
2849,C:\Users\Majid\Programming files\FYP\Datasets\...,1.920,TESS,happy
2393,C:\Users\Majid\Programming files\FYP\Datasets\...,2.566,TESS,sad
1122,C:\Users\Majid\Programming files\FYP\Datasets\...,2.444,TESS,sad
5194,C:/Users/Majid/Programming files/FYP/Datasets/...,3.403,RAVDESS,surprise
6215,C:/Users/Majid/Programming files/FYP/Datasets/...,3.470,RAVDESS,happy


Exploratory Data Analysis of df

In [6]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7332 entries, 0 to 7331
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   path      7332 non-null   object 
 1   duration  7332 non-null   float64
 2   dataset   7332 non-null   object 
 3   emotion   7332 non-null   object 
dtypes: float64(1), object(3)
memory usage: 229.2+ KB


In [7]:
import matplotlib.pyplot as plt
# Creating a figure with 2 subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 5))

# Check samples distribution
df.groupby(['emotion','dataset']).size().unstack().plot(kind='bar', stacked=True, ax=axes[0])
axes[0].set_title('Distribution of audio files by target emotion and dataset', size=14)
axes[0].set_ylabel('number of samples')
axes[0].legend(title='Dataset')

# Check duration distribution by each source using violin plots
sns.violinplot(x=df['dataset'],y=df['duration'], linewidth=1, ax=axes[1])
axes[1].set_xlabel('Emotion')
axes[1].set_ylabel('Samples duration (seconds)')
axes[1].set_title('Audio samples duration distribution for each emotion', size=14)

plt.show()

AttributeError: module 'matplotlib' has no attribute 'axes'

<Figure size 2000x500 with 0 Axes>

AttributeError: module 'matplotlib' has no attribute 'pyplot'

Had to drop digust because we cant really relate any specific emotion on the bases of recommending music playlist. As for the calm label we didnt have enough data to trarin


Average dataset for each emotion is from 1200+ to 1000+

Finally, now let's look at what is inside the audio files, picking some random samples from different targets. In particular I want to check:

the waveforms
the spectograms, with the fundamental frequency
the Mel-frequency cepstrum components (MFCCs), as "two-dimensional images".
and of course hear how they sound, using the built-in media player of python

In [ ]:
# function to display samples information by emotion
# note that a random samples is generated each time the function is called
# this is on purpose as to check different samples of each emotion every time

def show_audio(emotion):
    # create sublots
    fig, axs = plt.subplots(nrows=1,ncols=3, figsize=(20,4))
    # filter dataframe to emotion)
    df_show = df.loc[df['emotion'] == emotion].reset_index(drop=True)
    index = random.randint(0, df_show.shape[0])
    
    # load audio file:
    y, sr = librosa.load(df_show.path[index], sr=16000)
    
    # Show waveform
    librosa.display.waveshow(y, sr=sr, ax=axs[0])
    axs[0].set_title('Waveform')
    
    # Extract fundamental frequency (f0) using a probabilistic approach
    f0, _, _ = librosa.pyin(y, sr=sr, fmin=50, fmax=1500, frame_length=2048)

    # Establish timepoint of f0 signal
    timepoints = np.linspace(0, df_show.duration[index], num=len(f0), endpoint=False)
    
    # Compute short-time Fourier Transform
    x_stft = np.abs(librosa.stft(y))
    
    # Apply logarithmic dB-scale to spectrogram and set maximum to 0 dB
    x_stft = librosa.amplitude_to_db(x_stft, ref=np.max)
    
    # Plot STFT spectrogram
    librosa.display.specshow(x_stft, sr=sr, x_axis="time", y_axis="log", ax=axs[1])
    
    # Plot fundamental frequency (f0) in spectrogram plot
    axs[1].plot(timepoints, f0, color="cyan", linewidth=4)
    axs[1].set_title('Spectrogram with fundamental frequency')
    
    # Extract 'n_mfcc' numbers of MFCCs components - in this case 30
    x_mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)

    # Plot MFCCs
    librosa.display.specshow(x_mfccs, sr=sr, x_axis="time", norm=Normalize(vmin=-50, vmax=50), ax=axs[2])
    axs[2].set_title('MFCCs')
    
    # Show metadata in title
    plt.suptitle('File: {}  -  Emotion: {}'.format(df_show.path[index], df_show.emotion[index]), size=14)
    plt.tight_layout()
    plt.show()
    
    # Display media player for the selected file
    display(ipd.Audio(y, rate=sr))

In [ ]:
emotions = sorted(list(df.emotion.unique()))

# Get waveforms, spectograms, mfccs and media player for each emotion
for emotion in emotions:
    show_audio(emotion)

In [ ]:
# Create a variable where to store the mfccs data
mfccs = []

for file in df.path:
    # load audio file:
    y, sr = librosa.load(file, sr=16000)
    
    # Extract 'n_mfcc' numbers of MFCCs components - in this case 30
    mfccs.append(librosa.feature.mfcc(y, sr=sr, fmin=50, n_mfcc=30))

In [ ]:
# Define function to resize the 2D arrays
def resize_array(array):
    new_matrix = np.zeros((30,150))   # Initialize the new matrix shape with an array 30X150 of zeros
    for i in range(30):               # Iterate rows
        for j in range(150):          # Iterate columns
            try:                                 # the mfccs of a sample will replace the matrix of zeros, then cutting the array up to 150
                new_matrix[i][j] = array[i][j]
            except IndexError:                   # if mfccs of a sample is shorter than 150, then keep looping to extend lenght to 150 with 0s
                pass
    return new_matrix

# Create a variable to store the new resized mfccs and apply function for all the extracted mfccs
resized_mfccs = []

for mfcc in mfccs:
    resized_mfccs.append(resize_array(mfcc))


This code defines a function called resize_array which takes a 2D numpy array as input and returns a resized version of the array with shape (30, 150). The function first initializes a new matrix with shape (30, 150) filled with zeros. It then iterates over the rows and columns of the input array and attempts to copy the values into the new matrix. If the input array is shorter than 150 columns or 30 rows, the remaining entries in the new matrix are left as zeros. Finally, the resized matrix is appended to a list called resized_mfccs.

The code then iterates over a list called mfccs and applies the resize_array function to each array in the list, storing the resulting resized arrays in the resized_mfccs list.

The purpose of this code is to resize a set of 2D numpy arrays, which are likely extracted from audio data and contain mel frequency cepstral coefficients (MFCCs) used for speech and sound recognition. The resized arrays are then likely to be used as inputs to a neural network model for classification or regression.

In [ ]:
# Create sublots
fig, axs = plt.subplots(nrows=1,ncols=6, figsize=(20,3))

# Select 6 random MFCCs
for i in range(6):
    index = random.randint(0, len(resized_mfccs))
    
    # Plot MFCCs
    librosa.display.specshow(resized_mfccs[index], sr=sr, x_axis="time", ax=axs[i], norm=Normalize(vmin=-50, vmax=50))
    axs[i].set_title(str(df.emotion[index]) + ' - ' + str(df.duration[index]) + ' sec')

plt.suptitle('Few MFCCs of size 30x150', size=18)
plt.tight_layout()
plt.show()

Great, the reshaping function worked as expected, cutting the MFCC spectrum of the longer samples to ~4.5 seconds and adding silence for the shorter files to reach the same length.


Convolutional Neural Network model!

Let's split the data into train, validation and test set:

In [ ]:
from sklearn.model_selection import train_test_split

# Select target
df['emotion'].replace({'angry':0,'fear':1,'happy':2,'neutral':3,'sad':4,'surprise':5}, inplace=True)
y = df.emotion.values

# Features
X = resized_mfccs.copy()

# Create train, validation and test set
x_tr, x_te, y_tr, y_te = train_test_split(X, y, train_size=0.3, shuffle=True, random_state=0)
x_tr, x_va, y_tr, y_va = train_test_split(x_tr, y_tr, test_size=0.7, shuffle=True, random_state=0)

# Convert data to numpy arrays
x_tr = np.array([i for i in x_tr])
x_va = np.array([i for i in x_va])
x_te = np.array([i for i in x_te])

# Plot size of data
print(x_tr.shape)
print(x_va.shape)
print(x_te.shape)

In [ ]:
tr_mean = np.mean(x_tr, axis=0)
tr_std = np.std(x_tr, axis=0)

# Apply data scaling
x_tr = (x_tr - tr_mean)/tr_std
x_va = (x_va - tr_mean)/tr_std
x_te = (x_te - tr_mean)/tr_std

In [ ]:
print(f"x_tr has a dimension of {x_tr.shape} before the manipulation.")

x_tr = x_tr[..., None]
x_va = x_va[..., None]
x_te = x_te[..., None]

print(f"x_tr has a dimension of {x_tr.shape} after the manipulation.")

In [ ]:
x_tr = np.resize(x_tr, (len(x_tr), 30, 150, 1))
x_va = np.resize(x_va, (len(x_va), 30, 150, 1))
x_te = np.resize(x_te, (len(x_te), 30, 150, 1))


In [ ]:
%%time

from tensorflow import keras
from tensorflow.keras.layers import (Conv2D, BatchNormalization, Dropout, Flatten, Dense, MaxPool2D)
from tensorflow.keras import initializers

# Create convolutional neural network and return summary
model = keras.Sequential()
model.add(Conv2D(filters=64, kernel_size=5, strides=(2, 2), activation="relu", input_shape=x_tr.shape[1:]))
model.add(MaxPool2D(pool_size=2))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=4, strides=(2, 1), activation="relu"))
model.add(MaxPool2D(pool_size=2))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(units=7, activation="softmax"))
model.summary()

In [ ]:
# Compile the model using Adam's default learning rate
model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Create 'EarlyStopping' callback
earlystopping_cb = keras.callbacks.EarlyStopping(patience=5)

In [ ]:
#  Train the neural network
history = model.fit(
    x=x_tr,
    y=y_tr,
    epochs=100,
    batch_size=32,
    validation_data=(x_va, y_va),
    callbacks=[earlystopping_cb]
)


In [ ]:
model.save('model.h5')


In [ ]:
# Plots neural network performance metrics for train and validation
fig, axs = plt.subplots(1, 2, figsize=(20, 5))
plt.suptitle('Convolutional Netural Network with MFCCs', size=15)
results = pd.DataFrame(history.history)
results[["loss", "val_loss"]].plot(ax=axs[0])
axs[0].set_title("Validation loss {:.3f} (mean last 3)".format(np.mean(history.history["val_loss"][-3:])))
results[["accuracy", "val_accuracy"]].plot(ax=axs[1])
axs[1].set_title("Validation accuracy {:.3f} (mean last 3)".format(np.mean(history.history["val_accuracy"][-3:])))
plt.show()

In [ ]:
loss_te, accuracy_te = model.evaluate(x_te, y_te)

print("Test loss: {:.2f}".format(loss_te))
print("Test accuracy: {:.2f}%".format(100 * accuracy_te))

In [ ]:
# Compute test set predictions
predictions = model.predict(x_te)

pred = []

for i in predictions:
    pred.append(np.argmax(i))
    
from sklearn.metrics import ConfusionMatrixDisplay

labels = {'angry':0,'fear':1,'happy':2,'neutral':3,'sad':4,'surprise':5}

def plot_confusion_matrices(y_true, y_pred):

    # Create two subplots
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

    # Plots the standard confusion matrix
    ax1.set_title("Confusion Matrix (counts)")
    ConfusionMatrixDisplay.from_predictions(y_true, y_pred, display_labels=labels, ax=ax1)

    # Plots the normalized confusion matrix
    ax2.set_title("Confusion Matrix (ratios)")
    ConfusionMatrixDisplay.from_predictions(y_true, y_pred, display_labels=labels, normalize="true", ax=ax2)

    plt.show()

# Plot confusion matrices
plot_confusion_matrices(y_te, pred)